# <p style="text-align: center;"> AP EKF position and velocity estimation

In [1]:
import sympy as sp
import numpy as np
from scipy import linalg
from sympy import *
from IPython.display import Math, Latex

## List of tuning parameters for position and velocity fusion
#### A. Process noise parameters
1. $ACC\_P\_NSE$ : Accelerometer noise [m/s/s] $(AP \; value: 0.6)$ <br>
&emsp; <b>AP definition:</b> This control disturbance noise controls the growth of estimated error due to accelerometer measurement errors excluding bias. Increasing it makes the flter trust the accelerometer measurements less and other measurements more. <br>
<font color = red> <i><b>Interpretation:</b> This parameter dictates the amount of deviation from the motion model (of acceleration) that the estimated model (of acceleration) can have, i.e., whether the estimated model (of acceleration) will follow the motion model (of acceleration) or the true values (of acceleration).</i> </font> <br>
<font color = red> <i><b>Increasing the parameter:</b> The estimated model (of acceleration) will follow the true model (of acceleration) and will be more sensitive to noisy measurements.</i> </font> <br>
<font color = red> <i><b>Decreasing the parameter:</b> The estimated model (of acceleration) will follow the motion model (of acceleration), will be less sensitive to noisy measurements, and rapid changes in the true model (of acceleration) will be ignored (which should actually be tracked.).</i> </font> <br>
</br>
2. $ABIAS\_P\_NSE$ : Accelerometer bias stability [m/s/s/s] $(AP \; value: 0.005)$ <br>
&emsp; <b>AP definition:</b> This noise controls the growth of the vertical accelerometer delta velocity bias state error estimate. Increasing it makes accelerometer bias estimation faster and noisier. <br>
<font color = red> <i><b>Interpretation:</b> This parameter dictates the amount of deviation from the motion model (of accelerometer bias) that the estimated model (of accelerometer bias) can have, i.e., whether the estimated model (of accelerometer bias) will follow the motion model (of accelerometer bias) or the true values (of accelerometer bias).</i> </font> <br>
<font color = red> <i><b>Increasing the parameter:</b> The estimated model (of accelerometer bias) will follow the true model (of accelerometer bias) and will be more sensitive to noisy measurements.</i> </font> <br>
<font color = red> <i><b>Decreasing the parameter:</b> The estimated model (of accelerometer bias) will follow the motion model (of accelerometer bias), will be less sensitive to noisy measurements, and rapid changes in the true model (of accelerometer bias) will be ignored (which should actually be tracked.).</i> </font>

#### B. Measurement uncertainty parameters
3. $POSNE\_M\_NSE$ : GPS horizontal position measurement noise [m] $(AP \; value: 1)$ <br>
&emsp; <b>AP definition:</b> This sets the GPS horizontal position observation noise. Increasing it reduces the weighting of GPS horizontal position measurements. <br>
<font color = red> <i><b>Interpretation:</b> This parameter denotes the horizontal position measurement accuracy of the GPS sensor. It dictates whether the estimated model (of horizontal position) will follow the motion model (of horizontal position) or the true measurements (of horizontal position).</i> </font> <br>
<font color = red> <i><b>Increasing the parameter:</b> The estimated model (of horizontal position) will favor the motion model (of horizontal position) and rapid changes in the true measurements (of horizontal position) will be ignored.</i> </font> <br>
<font color = red> <i><b>Decreasing the parameter:</b> The estimated model (of horizontal position) will favor the true measurements (of horizontal position).</i> </font> <br>
</br>
4. $VELNE\_M\_NSE$ : GPS horizontal velocity measurement noise [m/s] $(AP \; value: 0.5)$ <br>
&emsp; <b>AP definition:</b> This sets a lower limit on the speed accuracy reported by the GPS receiver that is used to set horizontal velocity observation noise. If the model of receiver used does not provide a speed accurcy estimate, then the parameter value will be used. Increasing it reduces the weighting of the GPS horizontal velocity measurements. <br>
<font color = red> <i><b>Interpretation:</b> This parameter denotes the horizontal velocity measurement accuracy of the GPS sensor. It dictates whether the estimated model (of horizontal velocity) will follow the motion model (of horizontal velocity) or the true measurements (of horizontal velocity).</i> </font> <br>
<font color = red> <i><b>Increasing the parameter:</b> The estimated model (of horizontal velocity) will favor the motion model (of horizontal velocity) and rapid changes in the true measurements (of horizontal velocity) will be ignored.</i> </font> <br>
<font color = red> <i><b>Decreasing the parameter:</b> The estimated model (of horizontal velocity) will favor the true measurements (of horizontal velocity).</i> </font> <br>
</br>
5. $gpsHgtAccuracy$ : estimated height accuracy returned by the GPS receiver [m] $(AP \; value: 0)$ <br>
<font color = red> <i><b>Interpretation:</b> This parameter denotes the altitude measurement accuracy of the sensor. It dictates whether the estimated model (of altitude) will follow the motion model (of altitude) or the true measurements (of altitude).</i> </font> <br>
<font color = red> <i><b>Increasing the parameter:</b> The estimated model (of altitude) will favor the motion model (of altitude) and rapid changes in the true measurements (of altitude) will be ignored.</i> </font> <br>
<font color = red> <i><b>Decreasing the parameter:</b> The estimated model (of altitude) will favor the true measurements (of altitude).</i> </font>
</br>
6. $RNG\_M\_NSE$ : Range finder measurement noise [m] $(AP \; value: 0.5)$ <br>
&emsp; <b>AP definition:</b> This is the RMS value of noise in the range finder measurement. Increasing it reduces the weighting on this measurement. <br>
<font color = red> <i><b>Interpretation:</b> This parameter denotes the altitude measurement accuracy of the Altimeter. It dictates whether the estimated model (of altitude) will follow the motion model (of altitude) or the true measurements (of altitude).</i> </font> <br>
<font color = red> <i><b>Increasing the parameter:</b> The estimated model (of altitude) will favor the motion model (of altitude) and rapid changes in the true measurements (of altitude) will be ignored.</i> </font> <br>
<font color = red> <i><b>Decreasing the parameter:</b> The estimated model (of altitude) will favor the true measurements (of altitude).</i> </font>
</br>
7. $ALT\_M\_NSE$ : Baro height measurement noise [m] $(AP \; value: 3)$ <br>
&emsp; <b>AP definition:</b> This is the RMS value of noise in the altitude measurement. Increasing it reduces the weighting of the baro measurement and will make the filter respond more slowly to baro measurement errors, but will make it more sensitive to GPS and accelerometer errors. <br>
<font color = red> <i><b>Interpretation:</b> This parameter denotes the altitude measurement accuracy of the Barometer. It dictates whether the estimated model (of altitude) will follow the motion model (of altitude) or the true measurements (of altitude).</i> </font> <br>
<font color = red> <i><b>Increasing the parameter:</b> The estimated model (of altitude) will favor the motion model (of altitude) and rapid changes in the true measurements (of altitude) will be ignored.</i> </font> <br>
<font color = red> <i><b>Decreasing the parameter:</b> The estimated model (of altitude) will favor the true measurements (of altitude).</i> </font>
</br>
8. $VELD\_M\_NSE$ : GPS vertical velocity measurement noise [m/s] $(AP \; value: 0.5)$ <br>
&emsp; <b>AP definition:</b> This sets a lower limit on the speed accuracy reported by the GPS receiver that is used to set vertical velocity observation noise. If the model of receiver used does not provide a speed accurcy estimate, then the parameter value will be used. Increasing it reduces the weighting of the GPS vertical velocity measurements. <br>
<font color = red> <i><b>Interpretation:</b> This parameter denotes the vertical velocity measurement accuracy of the GPS sensor. It dictates whether the estimated model (of vertical velocity) will follow the motion model (of vertical velocity) or the true measurements (of vertical velocity).</i> </font> <br>
<font color = red> <i><b>Increasing the parameter:</b> The estimated model (of vertical velocity) will favor the motion model (of vertical velocity) and rapid changes in the true measurements (of vertical velocity) will be ignored.</i> </font> <br>
<font color = red> <i><b>Decreasing the parameter:</b> The estimated model (of vertical velocity) will favor the true measurements (of vertical velocity).</i> </font>

#### C. Innovation consistency check parameters
9. $POS\_I\_GATE$ : Percentage number of standard deviations applied to GPS position innovation consistency check [%] $(AP \; value: 500)$ <br>
&emsp; <b>AP definition:</b> This sets the percentage number of standard deviations applied to the GPS position measurement innovation consistency check. Decreasing it makes it more likely that good measurements will be rejected. Increasing it makes it more likely that bad measurements will be accepted. <br>
<font color = red> <i><b>Interpretation:</b> This parameter sets the margin within which if the difference between the estimated and measured horizontal positions falls, fusion of the horizontal positions will be successful.</i> </font> <br>
<font color = red> <i><b>Increasing the parameter:</b> The margin will be wider making it possible that even larger differences between the estimated and measured horizontal positions are accepted for fusion. Bad measurements may be accepted in this way.</i> </font> <br>
<font color = red> <i><b>Decreasing the parameter:</b> The margin will be narrower, allowing only small differences between the estimated and measured horizontal positions to be accepted for fusion. Good measurements may be rejected in this way.</i> </font> <br>
</br>
10. $HGT\_I\_GATE$ : Percentage number of standard deviations applied to height innovation consistency check [%] $(AP \; value: 500)$ <br>
&emsp; <b>AP definition:</b> This sets the percentage number of standard deviations applied to the height measurement innovation consistency check. Decreasing it makes it more likely that good measurements will be rejected. Increasing it makes it more likely that bad measurements will be accepted. <br>
<font color = red> <i><b>Interpretation:</b> This parameter sets the margin within which if the difference between the estimated and measured altitudes falls, fusion of the altitudes will be successful.</i> </font> <br>
<font color = red> <i><b>Increasing the parameter:</b> The margin will be wider making it possible that even larger differences between the estimated and measured altitudes are accepted for fusion. Bad measurements may be accepted in this way.</i> </font> <br>
<font color = red> <i><b>Decreasing the parameter:</b> The margin will be narrower, allowing only small differences between the estimated and measured altitudes to be accepted for fusion. Good measurements may be rejected in this way.</i> </font> <br>
</br>
11. $VEL\_I\_GATE$ : Percentage number of standard deviations applied to GPS velocity innovation consistency check [%] $(AP \; value: 500)$ <br>
&emsp; <b>AP definition:</b> This sets the percentage number of standard deviations applied to the GPS velocity measurement innovation consistency check. Decreasing it makes it more likely that good measurements willbe rejected. Increasing it makes it more likely that bad measurements will be accepted. <br>
<font color = red> <i><b>Interpretation:</b> This parameter sets the margin within which if the difference between the estimated and measured horizontal velocities falls, fusion of the velocities will be successful.</i> </font> <br>
<font color = red> <i><b>Increasing the parameter:</b> The margin will be wider making it possible that even larger differences between the estimated and measured velocities are accepted for fusion. Bad measurements may be accepted in this way.</i> </font> <br>
<font color = red> <i><b>Decreasing the parameter:</b> The margin will be narrower, allowing only small differences between the estimated and measured velocities to be accepted for fusion. Good measurements may be rejected in this way.</i> </font> <br>
</br>
12. $FS\_EKF\_THRESH$ : EKF failsafe variance threshold $(AP \; value: 0.8)$ <br>
&emsp; <b>AP definition:</b> Allows setting the maximum acceptable compass, position, and velocity variance. <br>
<font color = red> <i><b>Interpretation:</b> This parameter can be thought of as the RMS value of the innovations of the horizontal positions / horizontal velocities / altitudes. While the GATE parameter defines a margin to accept or reject (fluctuating) measurements, the FS_EKF_THRESH parameter defines a maximum acceptable limit on the stable or RMS values of the innovations, exceeding which will result in an EKF failsafe action. A balance has to be struck between choosing the two parameters so as to satisfy the margin as well as the limit.</i> </font> <br>
<font color = red> <i><b>Increasing the parameter:</b> In the worst case scenario, having a high GATE parameter (s.t. for e.g. posTestRatio $<$ 1) and also increasing the FS_EKF_THRESH parameter will result in bad measurements with high RMS values to be accepted, therefore resulting in a reduced estimation accuracy.</i> </font> <br>
<font color = red> <i><b>Decreasing the parameter:</b> In the worst case scenario, having a low GATE parameter (s.t. for e.g. posTestRatio $<$ 1) and also decreasing the FS_EKF_THRESH parameter will result in good measurements with low RMS values to be rejected, therefore resulting in an EKF failsafe action.</i> </font>

## System model

The system model is represented in the form of the state extrapolation equation: <br>
<p style="text-align: center;"> $X_{t|t-1} = F.X_{t-1|t-1} + G.U_{t-1|t-1}$ <br> </p>

where, <br>
$X_{t|t-1}$ = Predicted system state at time t <br>
$F$ = State transition matrix <br>
$X_{t-1|t-1}$ = System state at time t-1 <br>
$G$ = Control matrix <br>
$U_{t-1|t-1}$ = Control vector at time t-1

In [3]:
# XT_predict_compact = Predicted system state at time t
# F = State transition matrix
# XT_previous = System state at time t-1
# G = Control matrix
# U = Control vector at time t-1

pos_xt1, vel_xt1, pos_yt1, vel_yt1, pos_zt1, vel_zt1, delt, pos_xt, vel_xt, pos_yt, vel_yt, pos_zt, vel_zt, acc_xt, acc_yt, acc_zt = sp.symbols('x_{t|t-1}, \dot{x}_{t|t-1}, y_{t|t-1}, \dot{y}_{t|t-1}, z_{t|t-1}, \dot{z}_{t|t-1}, \Delta{t}, x_{t-1}, \dot{x}_{t-1}, y_{t-1}, \dot{y}_{t-1}, z_{t-1}, \dot{z}_{t-1}, \ddot{x}_{t-1}, \ddot{y}_{t-1}, \ddot{z}_{t-1}')

Xt_predict_compact = sp.Matrix([[pos_xt1],[vel_xt1],[pos_yt1],[vel_yt1],[pos_zt1],[vel_zt1]])
F = sp.Matrix([[1,delt,0,0,0,0],[0,1,0,0,0,0],[0,0,1,delt,0,0],[0,0,0,1,0,0],[0,0,0,0,1,delt],[0,0,0,0,0,1]])
Xt_previous = sp.Matrix([[pos_xt],[vel_xt],[pos_yt],[vel_yt],[pos_zt],[vel_zt]])
G = sp.Matrix([[delt**2/2,0,0],[delt,0,0],[0,delt**2/2,0],[0,delt,0],[0,0,delt**2/2],[0,0,delt]])
Ut_previous = sp.Matrix([[acc_xt],[acc_yt],[acc_zt]])

display(Math(r'X_{{t|t-1}} = {}, F = {}, X_{{t-1|t-1}} = {}, G = {}, U_{{t-1|t-1}} = {}'.format(sp.latex(Xt_predict_compact), sp.latex(F), sp.latex(Xt_previous), sp.latex(G), sp.latex(Ut_previous))))

<IPython.core.display.Math object>

Arranging the matrices in the form of the state extrapolation equation: $X_{t|t-1} = F.X_{t-1|t-1} + G.U_{t-1|t-1}$

In [4]:
display(Math(r'{} = {}{} + {}{}'.format(sp.latex(Xt_predict_compact), sp.latex(F), sp.latex(Xt_previous), sp.latex(G), sp.latex(Ut_previous))))

<IPython.core.display.Math object>

## Predicted system state $X$ (at time t-1, for time t)

In [5]:
Xt_predict = F*Xt_previous+G*Ut_previous

display(Math(r'X_{{t|t-1}} = {} = {}'.format(sp.latex(Xt_predict_compact), sp.latex(Xt_predict))))

<IPython.core.display.Math object>

## Process noise (due to accelerometer measurements)

In [13]:
# N = Process noise matrix
# Q = Process noise covariance matrix

N_acc_x, N_acc_y, N_acc_z = sp.symbols('q_{\ddot{x}^2}, q_{\ddot{y}^2}, q_{\ddot{z}^2}')
N = sp.Matrix([[N_acc_x,0,0],[0,N_acc_y,0],[0,0,N_acc_z]])
Q = G*N*G.T

display(Math(r'N = {}'.format(sp.latex(N))))
display(Math(r'Q = {}'.format(sp.latex(Q))))

<IPython.core.display.Math object>

<IPython.core.display.Math object>

The process noise matrix and its variables in the AP code are as follows:

In [3]:
N_acc_x_AP, N_acc_y_AP, N_acc_z_AP = sp.symbols('(accNoise)^2, (accNoise)^2, (accNoise)^2')

N_AP = sp.Matrix([[N_acc_x_AP,0,0],[0,N_acc_y_AP,0],[0,0,N_acc_z_AP]])

display(Math(r'N_{{AP}} = {}'.format(sp.latex(N_AP))))
display(Math(r'accNoise = ACC\_P\_NSE'))

<IPython.core.display.Math object>

<IPython.core.display.Math object>

## Process noise (due to accelerometer bias)

In [22]:
# B = Accelerometer bias process noise matrix

B_acc_x, B_acc_y, B_acc_z = sp.symbols('b_{\ddot{x}^2}, b_{\ddot{y}^2}, b_{\ddot{z}^2}')
B = sp.Matrix([[0,0,0,0,0,0],[0,B_acc_x,0,0,0,0],[0,0,0,0,0,0],[0,0,0,B_acc_y,0,0],[0,0,0,0,0,0],[0,0,0,0,0,B_acc_z]])

display(Math(r'B = {}'.format(sp.latex(B))))

<IPython.core.display.Math object>

The process noise matrix and its variables in the AP code are as follows:

In [5]:
B_acc_x_AP, B_acc_y_AP, B_acc_z_AP = sp.symbols('(accelBiasProcessNoise*\Delta{t}^2)^2, (accelBiasProcessNoise*\Delta{t}^2)^2, (accelBiasProcessNoise*\Delta{t}^2)^2')
B_AP = sp.Matrix([[0,0,0,0,0,0],[0,B_acc_x_AP,0,0,0,0],[0,0,0,0,0,0],[0,0,0,B_acc_y_AP,0,0],[0,0,0,0,0,0],[0,0,0,0,0,B_acc_z_AP]])

display(Math(r'B_{{AP}} = {}'.format(sp.latex(B_AP))))
display(Math(r'accelBiasProcessNoise = ABIAS\_P\_NSE'))

<IPython.core.display.Math object>

<IPython.core.display.Math object>

## Measurement uncertainty

In [8]:
# R = Measurement uncertainty covariance matrix
# r = Measurement uncertainty standard deviation (sigma) of each sensor that can measure a system state variable

R_pos_x, R_vel_x, R_pos_y, R_vel_y, R_pos_z, R_vel_z = sp.symbols('r_{x}^2, r_{\dot{x}}^2, r_{y}^2, r_{\dot{y}}^2, r_{z}^2, r_{\dot{z}}^2')

R = sp.Matrix([[R_pos_x,0,0,0,0,0],[0,R_vel_x,0,0,0,0],[0,0,R_pos_y,0,0,0],[0,0,0,R_vel_y,0,0],[0,0,0,0,R_pos_z,0],[0,0,0,0,0,R_vel_z]])

display(Math(r'R = {}'.format(sp.latex(R))))

<IPython.core.display.Math object>

The measurement uncertainty covariance matrix and its variables in the AP code are as follows:

In [4]:
R_pos_x_AP, R_vel_x_AP, R_pos_y_AP, R_vel_y_AP, R_pos_z_AP, R_vel_z_AP = sp.symbols('{(gpsHorizPosAccuracy)}^2, {(gpsHorizVelAccuracy)}^2, {(gpsHorizPosAccuracy)}^2, {(gpsHorizVelAccuracy)}^2, posDownObsNoise, {(gpsVertVelAccuracy)}^2')

R_AP = sp.Matrix([[R_pos_x_AP,0,0,0,0,0],[0,R_vel_x_AP,0,0,0,0],[0,0,R_pos_y_AP,0,0,0],[0,0,0,R_vel_y_AP,0,0],[0,0,0,0,R_pos_z_AP,0],[0,0,0,0,0,R_vel_z_AP]])

display(Math(r'R_{{AP}} = {}'.format(sp.latex(R_AP))))

<IPython.core.display.Math object>

$gpsHorizPosAccuracy = POSNE\_M\_NSE$ <br>
$gpsHorizVelAccuracy = VELNE\_M\_NSE$ <br>
$gpsVertVelAccuracy = VELD\_M\_NSE$ <br>
#### Note on $posDownObsNoise$:
If height measurement source = GPS <br>
&emsp; $posDownObsNoise = (gpsHgtAccuracy)^2$ <br>
Else <br>
If height measurement source = Rangefinder <br>
&emsp; $posDownObsNoise = (rngNoise)^2 = (RNG\_M\_NSE)^2$ <br>
Else <br>
If height measurement source = Barometer <br>
&emsp; $posDownObsNoise = (baroAltNoise)^2 = (ALT\_M\_NSE)^2$

## Initialized estimation uncertainty covariance matrix $P$ (at time t-1, for time t-1)

In [10]:
# Pt_previous = Estimated estimation uncertainty covariance matrix of the system state variables at time t-1

P_x, P_xdot, P_y, P_ydot, P_z, P_zdot = sp.symbols('P_x|t-1|t-1, P_{\dot{x}|t-1|t-1}, P_y|t-1|t-1, P_{\dot{y}|t-1|t-1}, P_z|t-1|t-1, P_{\dot{z}|t-1|t-1}')

Pt_previous = sp.Matrix([[P_x,0,0,0,0,0],[0,P_xdot,0,0,0,0],[0,0,P_y,0,0,0],[0,0,0,P_ydot,0,0,],[0,0,0,0,P_z,0],[0,0,0,0,0,P_zdot]])

display(Math(r'P_{{t-1|t-1}} = {}'.format(sp.latex(Pt_previous))))

<IPython.core.display.Math object>

The initialized estimation uncertainty covariance matrix and its variables in the AP code are as follows:

In [5]:
P_x_AP, P_xdot_AP, P_y_AP, P_ydot_AP, P_z_AP, P_zdot_AP = sp.symbols('{(gpsHorizPosAccuracy)}^2, {(gpsHorizVelAccuracy)}^2, {(gpsHorizPosAccuracy)}^2, {(gpsHorizVelAccuracy)}^2, posDownObsNoise, {(gpsVertVelAccuracy)}^2')

Pt_previous_AP = sp.Matrix([[P_x_AP,0,0,0,0,0],[0,P_xdot_AP,0,0,0,0],[0,0,P_y_AP,0,0,0],[0,0,0,P_ydot_AP,0,0,],[0,0,0,0,P_z_AP,0],[0,0,0,0,0,P_zdot_AP]])

display(Math(r'P_{{t-1|t-1|AP}} = {}'.format(sp.latex(Pt_previous_AP))))

<IPython.core.display.Math object>

$gpsHorizPosAccuracy = POSNE\_M\_NSE$ <br>
$gpsHorizVelAccuracy = VELNE\_M\_NSE$ <br>
$gpsVertVelAccuracy = VELD\_M\_NSE$ <br>
#### Note on $posDownObsNoise$:
If height measurement source = GPS <br>
&emsp; $posDownObsNoise = (gpsHgtAccuracy)^2$ <br>
Else <br>
If height measurement source = Rangefinder <br>
&emsp; $posDownObsNoise = (rngNoise)^2 = (RNG\_M\_NSE)^2$ <br>
Else <br>
If height measurement source = Barometer <br>
&emsp; $posDownObsNoise = (baroAltNoise)^2 = (ALT\_M\_NSE)^2$

## Predicted estimation uncertainty covariance matrix $P$ (at time t-1, for time t)

In [32]:
# Pt_predict = Predicted estimation uncertainty covariance matrix of the system state variables at time t
# Note: In AP, Pt_predict is first calculated and the accelerometer bias process noise matrix is added to its diagonals in the end.

Pt_predict = (F*Pt_previous*F.T + Q + B)

display(Math(r'P_{{t|t-1}} = {}'.format(sp.latex(Pt_predict))))

<IPython.core.display.Math object>

## Updated measurement vector (at time t, for time t)

In [15]:
# Zt_update = Measurement vector at time t containing the sensor measurements
# H = Observation matrix to map the measurement vector to the system state variables

pos_xt, vel_xt, pos_yt, vel_yt, pos_zt, vel_zt = sp.symbols('z_{x|t}, z_{\dot{x}|t}, z_{y|t}, z_{\dot{y}|t}, z_{z|t}, z_{\dot{z}|t}')

Zt_update = sp.Matrix([[pos_xt],[vel_xt],[pos_yt],[vel_yt],[pos_zt],[vel_zt]])
H = sp.Matrix([[1,0,0,0,0,0],[0,1,0,0,0,0],[0,0,1,0,0,0],[0,0,0,1,0,0],[0,0,0,0,1,0],[0,0,0,0,0,1]])

display(Math(r'Z_{{t|t}} = {}, H = {}'.format(sp.latex(Zt_update), sp.latex(H))))

<IPython.core.display.Math object>

The measurement vector and its variables in the AP code are as follows:

In [16]:
pos_xt_AP, vel_xt_AP, pos_yt_AP, vel_yt_AP, pos_zt_AP, vel_zt_AP = sp.symbols('posxy.x, gpsDataDelayed.vel.x, posxy.y, gpsDataDelayed.vel.y, gpsDataDelayed.hgt, gpsDataDelayed.vel.z')

Zt_update_AP = sp.Matrix([[pos_xt_AP],[vel_xt_AP],[pos_yt_AP],[vel_yt_AP],[pos_zt_AP],[vel_zt_AP]])
display(Math(r'Z_{{t|t|AP}} = {}'.format(sp.latex(Zt_update_AP))))

<IPython.core.display.Math object>

#### Note on $z_{z|t}$:
If height measurement source = GPS <br>
&emsp; $z_{z|t} = gpsDataDelayed.hgt$ <br>
Else <br>
If height measurement source = Rangefinder <br>
&emsp; $z_{z|t} = rangeDataDelayed.hgt$ <br>
Else <br>
If height measurement source = Barometer <br>
&emsp; $z_{z|t} = baroDataDelayed.hgt$

## Updated innovation matrix (at time t, for time t)

In [17]:
# Yt_update = Updated innovation matrix at time t

Yt_update = H*Xt_predict_compact - Zt_update

display(Math(r'Y_{{t|t}} = {}'.format(sp.latex(Yt_update))))

<IPython.core.display.Math object>

## Updated innovation covariance matrix (at time t, for time t)

In [33]:
# St_update = Updated innovation covariance matrix at time t

St_update = H*Pt_predict*H.T + R

display(Math(r'S_{{t|t}} = {}'.format(sp.latex(St_update))))

<IPython.core.display.Math object>

## Innovation consistency check on horizontal (x-y) positions

In [4]:
Yt_11, Yt_31, maxPosInnovXY = sp.symbols('Y_{t|t|11}, Y_{t|t|31}, maxPosInnovXY')

display(Math(r'maxPosInnovXY = max(0.01*gpsPosInnovGate, 1)^2 * (S_{{t|t|11}} + S_{{t|t|33}})'))
display(Math(r'gpsPosInnovGate = POS\_I\_GATE'))

posTestRatio = ((Yt_11)**2 + (Yt_31)**2) / maxPosInnovXY

display(Math(r'posTestRatio = {}'.format(sp.latex(posTestRatio))))

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

#### Condition for successful fusion of horizontal (x-y) positions data:
If (posTestRatio < 1) <br>
&emsp; fusePosData = True; <br>
Else <br>
&emsp; fusePosData = False;

If fusePosData = False, changes are made to the predicted estimation uncertainty matrix $P_{t|t-1}$ and the $posTestRatio$ so that fusion of the position data becomes successful in the further EKF steps.

#### EKF failsafe due to horizontal (x-y) position variance
If $\sqrt{posTestRatio} > FS\_EKF\_THRESH$ for two consecutive variances, an EKF failsafe will be triggered (1. Land; 2. AltHold; 3. Land even in stabilize).

## Innovation consistency check on vertical (z) position

In [5]:
Yt_51, maxPosInnovZ = sp.symbols('Y_{t|t|51}, maxPosInnovZ')

display(Math(r'maxPosInnovZ = max(0.01*hgtInnovGate, 1)^2 * S_{{t|t|55}}'))
display(Math(r'hgtInnovGate = HGT\_I\_GATE'))

hgtTestRatio = (Yt_51)**2 / maxPosInnovZ

display(Math(r'hgtTestRatio = {}'.format(sp.latex(hgtTestRatio))))

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

#### Condition for successful fusion of vertical (z) position data:
If (PV_AidingMode == AID_NONE && onGround) <br>
&emsp; maxTestRatio = 3; <br>
Else <br>
&emsp; maxTestRatio = 1; <br>
</br>
If (hgtTestRatio < maxTestRatio) <br>
&emsp; fuseHgtData = True; <br>
Else <br>
&emsp; fuseHgtData = False;

If fuseHgtData = False, changes are made to the predicted estimation uncertainty matrix $P_{t|t-1}$ and the $hgtTestRatio$ so that fusion of the position data becomes successful in the further EKF steps.

#### EKF failsafe due to vertical (z) position variance
If $\sqrt{hgtTestRatio} > FS\_EKF\_THRESH$ for two consecutive variances, an EKF failsafe will be triggered (1. Land; 2. AltHold; 3. Land even in stabilize).

## Innovation consistency check on (x-y-z) velocities

In [6]:
Yt_21, Yt_41, Yt_61, maxVelInnovXYZ = sp.symbols('Y_{t|t|21}, Y_{t|t|41}, Y_{t|t|61}, maxVelInnovXYZ')

display(Math(r'maxVelInnovXYZ = max(0.01*gpsVelInnovGate, 1)^2 * (S_{{t|t|22}} + S_{{t|t|44}} + S_{{t|t|66}})'))
display(Math(r'gpsVelInnovGate = VEL\_I\_GATE'))

velTestRatio = ((Yt_21)**2 + (Yt_41)**2 + (Yt_61)**2) / maxVelInnovXYZ

display(Math(r'velTestRatio = {}'.format(sp.latex(velTestRatio))))

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

#### Condition for successful fusion of (x-y-z) velocities data:
If (velTestRatio < 1) <br>
&emsp; fuseVelData = True; <br>
Else <br>
&emsp; fuseVelData = False;

If fuseVelData = False, changes are made to the predicted estimation uncertainty matrix $P_{t|t-1}$ and the $velTestRatio$ so that fusion of the velocity data becomes successful in the further EKF steps.

#### EKF failsafe due to (x-y-z) velocities variance
If $\sqrt{velTestRatio} > FS\_EKF\_THRESH$ for two consecutive variances, an EKF failsafe will be triggered (1. Land; 2. AltHold; 3. Land even in stabilize).

In [34]:
Num = sp.adjoint(St_update).simplify()

In [35]:
Den = sp.det(St_update).simplify()

In [36]:
St_inv = (Num/Den).simplify()

display(Math(r'S_{{t|t}}^{{-1}} = {}'.format(sp.latex(St_inv))))

<IPython.core.display.Math object>

## Updated Kalman gain $K$ (at time t, for time t)

In [24]:
# Kt_update = Updated Kalman gain at time t

Kt_update = (Pt_predict*H.T*St_inv).simplify()

display(Math(r'K_{{t|t}} = {}'.format(sp.latex(Kt_update))))

KeyboardInterrupt: 

## Estimated system state $X$ (at time t, for time t)

In [66]:
# Xt_estimate = Estimated system state at time t

Xt_estimate = (Xt_predict_compact + Kt_update*(Yt_update))

display(Math(r'X_{{t|t}} = {}'.format(sp.latex(Xt_estimate))))

<IPython.core.display.Math object>

## Estimated estimation uncertainty covariance matrix $P$ (at time t, for time t)

In [67]:
# Pt_estimate = Estimated estimation uncertainty matrix of the system state variables at time t

I = sp.Matrix([[1,0,0,0,0,0],[0,1,0,0,0,0],[0,0,1,0,0,0],[0,0,0,1,0,0],[0,0,0,0,1,0],[0,0,0,0,0,1]])

Pt_estimate = ((I - Kt_update*H)*Pt_predict)

display(Math(r'P_{{t|t}} = {}'.format(sp.latex(Pt_estimate))))

<IPython.core.display.Math object>